In [2]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=5d1e71f3d2fee7f9835be83815ea775544172f38c95ed974fc65fcccefad2e4d
  Stored in directory: /Users/huan/Library/Caches/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [1]:
import pandas as pd
import sqlite3
from langdetect import detect, LangDetectException
import sys

# Set pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
# load query dataset
QUERY_CSV_FILE = 'esci-data/shopping_queries_dataset_small.csv'
DATABASE_FILE = 'amz.db'

df_queries = pd.read_csv(QUERY_CSV_FILE)
print("Query dataset loaded successfully.")
print("First 5 rows:")
print(df_queries.head())
print("\nDataFrame Info:")
df_queries.info()

Query dataset loaded successfully.
First 5 rows:
   example_id                         query  query_id  product_id  \
0          16  !awnmower tires without rims         1  B075SCHMPY   
1          17  !awnmower tires without rims         1  B08L3B9B9P   
2          18  !awnmower tires without rims         1  B082K7V2GZ   
3          19  !awnmower tires without rims         1  B07P4CF3DP   
4          20  !awnmower tires without rims         1  B07C1WZG12   

  product_locale esci_label  small_version  large_version  split  
0             us          I              1              1  train  
1             us          E              1              1  train  
2             us          I              1              1  train  
3             us          S              1              1  train  
4             us          E              1              1  train  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118011 entries, 0 to 1118010
Data columns (total 9 columns):
 #   

In [3]:
import sqlite3
import pandas as pd

# Create a connection
try:
    conn = sqlite3.connect(DATABASE_FILE)
    print(f"Successfully connected to {DATABASE_FILE}")
except sqlite3.Error as e:
    print(f"Error connecting to database: {e}")

Successfully connected to amz.db


In [4]:
# Get Valid Product IDs from Database
valid_product_asins = set()
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT parent_asin FROM products")

rows = cursor.fetchall()
for row in rows:
    if row[0]: # Ensure it's not None
        valid_product_asins.add(row[0])
        
print(f"Loaded {len(valid_product_asins)} unique product ASINs from database.")

Loaded 1033025 unique product ASINs from database.


In [5]:
# Filter 1: Keep rows where 'product_id' is in our set of valid ASINs
initial_rows = len(df_queries)
df_filtered_1 = df_queries[df_queries['product_id'].isin(valid_product_asins)].copy()
filtered_rows_1 = len(df_filtered_1)

print(f"Filter 1 (Product ID):")
print(f"Original rows: {initial_rows}")
print(f"Rows after filtering: {filtered_rows_1}")
print(f"Rows removed: {initial_rows - filtered_rows_1}")

Filter 1 (Product ID):
Original rows: 1118011
Rows after filtering: 2900
Rows removed: 1115111


In [6]:
def is_english(text):
    """Returns True if text is detected as English, False otherwise."""
    if not isinstance(text, str) or not text.strip():
        return False  # Handle empty strings or non-string types
    try:
        # Detect the language
        if detect(text) == 'en':
            return True
        else:
            return False
    except LangDetectException:
        # Failed to detect (e.g., text is just punctuation or numbers)
        return False
    except Exception as e:
        print(f"Error detecting language for '{text}': {e}")
        return False
    

initial_rows_2 = len(df_filtered_1)

# Apply the language detection function to the 'query' column
df_filtered_1['is_english'] = df_filtered_1['query'].apply(is_english)

# Filter 2: Keep rows where 'is_english' is True
df_final_filtered = df_filtered_1[df_filtered_1['is_english'] == True].copy()

# We can drop the temporary 'is_english' column
df_final_filtered = df_final_filtered.drop(columns=['is_english'])

filtered_rows_2 = len(df_final_filtered)

print(f"Filter 2 (Language):")
print(f"Rows before language filter: {initial_rows_2}")
print(f"Rows after language filter: {filtered_rows_2}")
print(f"Rows removed: {initial_rows_2 - filtered_rows_2}")

print("\nFinal Filtered DataFrame (Head):")
print(df_final_filtered.head())

Filter 2 (Language):
Rows before language filter: 2900
Rows after language filter: 1492
Rows removed: 1408

Final Filtered DataFrame (Head):
     example_id                                              query  query_id  \
195         368  #1 best and not expensive bath back brush crea...        11   
560         765  #12 black boys chain necklace without baseball...        23   
580         801                                          #15 charm        25   
837        1074                     #4 braiding hair not stretched        33   
868        1105                     #4 braiding hair not stretched        33   

     product_id product_locale esci_label  small_version  large_version  split  
195  B07KXVGHTV             us          S              1              1  train  
560  B074WDYN1R             us          I              1              1  train  
580  B07VFDGXBY             us          S              1              1  train  
837  B08J7L9SRF             us          I             

In [7]:
# Get the list of unique product_id's from filtered queries
product_ids_of_interest = list(df_final_filtered['product_id'].unique())
placeholders = ','.join(['?'] * len(product_ids_of_interest))
sql_query = f"SELECT * FROM products WHERE parent_asin IN ({placeholders})"
df_products_of_interest = pd.read_sql_query(sql_query, conn, params=product_ids_of_interest)
            
print(f"\nSuccessfully selected {len(df_products_of_interest)} matching products from the database.")
print("\nProduct DataFrame Info:")
df_products_of_interest.info()


Successfully selected 1352 matching products from the database.

Product DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_id            1352 non-null   int64  
 1   parent_asin           1352 non-null   object 
 2   main_category         1352 non-null   object 
 3   product_title         1352 non-null   object 
 4   average_rating        1352 non-null   float64
 5   rating_number         1352 non-null   int64  
 6   features_json         1352 non-null   object 
 7   product_description   1352 non-null   object 
 8   price                 1352 non-null   object 
 9   images_json           1352 non-null   object 
 10  videos_json           1352 non-null   object 
 11  store                 1352 non-null   object 
 12  categories_json       1352 non-null   object 
 13  details_json          1352 non-nu

In [8]:
def is_value_empty(val):
    """Checks if a value is empty (None, NaN, '', '[]', '{}', 'null')."""
    if pd.isna(val):
        return True
    s_val = str(val).strip()
    if s_val == '' or s_val in ('[]', '{}') or s_val.lower() == 'null' or s_val.lower() == 'nan':
        return True
    return False

results = []
total_rows = len(df_products_of_interest)
for column in df_products_of_interest.columns:
    empty_count = df_products_of_interest[column].apply(is_value_empty).sum()
    
    # Calculate non-empty
    non_empty_count = total_rows - empty_count
    results.append({
        'Column': column,
        'Percentage': f"{(non_empty_count / total_rows * 100):.2f}%"
    })

# Convert results to a DataFrame for pretty printing
df_non_empty_counts = pd.DataFrame(results)
print(df_non_empty_counts.to_string(index=False))

              Column Percentage
          product_id    100.00%
         parent_asin    100.00%
       main_category     99.41%
       product_title    100.00%
      average_rating    100.00%
       rating_number    100.00%
       features_json     64.57%
 product_description     29.14%
               price     31.21%
         images_json    100.00%
         videos_json     37.13%
               store     99.26%
     categories_json     27.96%
        details_json     98.67%
bought_together_json      0.00%
               brand      0.00%
               color      0.00%
      product_locale      0.00%


In [9]:
print(df_products_of_interest['average_rating'].describe())
print(df_products_of_interest['rating_number'].describe())

count    1352.000000
mean        4.150444
std         0.556809
min         1.000000
25%         3.900000
50%         4.300000
75%         4.500000
max         5.000000
Name: average_rating, dtype: float64
count     1352.000000
mean       703.309911
std       3751.506926
min          1.000000
25%         20.000000
50%         76.500000
75%        312.750000
max      90203.000000
Name: rating_number, dtype: float64


###  filter selectivity analysis

In [10]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

def _extract_recursive(obj, values_list):
    """
    Recursively navigates a JSON object (dict or list) and
    appends all "leaf" string values to the values_list.
    """
    if isinstance(obj, dict):
        # If it's a dict, recurse on all its values
        for v in obj.values():
            _extract_recursive(v, values_list)
    elif isinstance(obj, list):
        # If it's a list, recurse on all its items
        for item in obj:
            _extract_recursive(item, values_list)
    elif obj is not None:
        # We've hit a "leaf" node (str, int, float, bool)
        val_str = str(obj).strip()
        if val_str: # Don't add empty strings
            values_list.append(val_str)

def get_json_values(json_string):
    """
    Safely parses a JSON string and returns a flat list of ALL
    string values, including those in nested lists or dicts.
    """
    if is_value_empty(json_string):
        return []
    
    values_to_return = []
    try:
        obj = json.loads(json_string)
        # Call the recursive helper to populate the list
        _extract_recursive(obj, values_to_return)
    except (json.JSONDecodeError, TypeError):
        print(f'Warning, bad JSON: {json_string}')
        return [] # Return empty on bad JSON
    
    return values_to_return

def get_filter_stats(cursor, where_clause, params, asin_to_check, total_products_in_db, use_fts=False):
    """
    Checks if a filter is effective (finds its own ASIN) and, if so,
    returns its selectivity (match count and percentage).
    
    Returns: A dict {'match_count': int, 'match_percentage': float} or None
    """
    # This function assumes errors are part of the analysis, so no new try/except
    # Effectiveness Check
    db_table = "products_fts" if use_fts else "products"
    check_sql = f"SELECT 1 FROM {db_table} WHERE ({where_clause}) AND parent_asin = ? LIMIT 1"
    check_params = params + (asin_to_check,)
    if cursor.execute(check_sql, check_params).fetchone() is None:
        print(f"Ineffective filter detected with SQL query: {check_sql}, params: {check_params}")
        return None # Filter is INEFFECTIVE
    
    # Selectivity Check
    count_sql = f"SELECT COUNT(*) FROM {db_table} WHERE ({where_clause})"
    count = cursor.execute(count_sql, params).fetchone()[0]
    percentage = round((count / total_products_in_db) * 100, 3)
    if percentage == 0:
        print(f"Ineffective filter detected (0% selectivity) with SQL query: {count_sql}, params: {params}")
        return None # Filter is INEFFECTIVE
    return {
        'match_count': count,
        'match_percentage': percentage
    }

filter_columns_config = {
    'main_category': {'type': 'exact_match'},
    'product_title': {'type': 'NER_contains'},
    'store': {'type': 'exact_match'},
    'average_rating': {'type': 'range'},
    'rating_number': {'type': 'range'},
    'price': {'type': 'range'},
    'features_json': {'type': 'json_contains'},
    'details_json': {'type': 'json_contains'}
}


In [11]:
print("Pre-fetching range column data for efficient delta calculation...")
range_columns = [col for col, conf in filter_columns_config.items() if conf['type'] == 'range']
distributions = {}
total_products_in_db = 0

try:
    total_products_in_db = cursor.execute("SELECT COUNT(*) FROM products").fetchone()[0]
    if total_products_in_db == 0:
        raise Exception("No products in database.")

    for col in range_columns:
        print(f"  Fetching all non-null '{col}' values...")
        # Fetch only non-null, valid numeric values
        df_dist = pd.read_sql_query(f"SELECT {col} FROM products WHERE {col} IS NOT NULL", conn)
        series_dist = pd.to_numeric(df_dist[col], errors='coerce').dropna()
        distributions[col] = np.sort(series_dist.values)
        print(f"  -> Fetched and sorted {len(distributions[col])} values for '{col}'.")
        
    print("Pre-fetching complete.")

except Exception as e:
    print(f"Error during pre-fetch: {e}. Cannot proceed with range calculations.")
    # Set distributions to empty to skip range logic
    distributions = {}
    total_products_in_db = 0

Pre-fetching range column data for efficient delta calculation...
  Fetching all non-null 'average_rating' values...
  -> Fetched and sorted 1033025 values for 'average_rating'.
  Fetching all non-null 'rating_number' values...
  -> Fetched and sorted 1033025 values for 'rating_number'.
  Fetching all non-null 'price' values...
  -> Fetched and sorted 114679 values for 'price'.
Pre-fetching complete.


In [12]:
print("Creating FTS5 virtual table 'products_fts'...")
# Create the virtual table. 
# include the columns we want to search and the parent_asin so we can validate filters.
cursor.execute("""
    CREATE VIRTUAL TABLE IF NOT EXISTS products_fts USING fts5(
        parent_asin, 
        features_json, 
        details_json,
        product_title,
    );
""")
print("Clearing FTS table to prevent duplicates...")
# Delete all existing data before inserting new data.
cursor.execute("DELETE FROM products_fts")
# Populate the FTS table from the products table
cursor.execute("""
    INSERT INTO products_fts (parent_asin, features_json, details_json, product_title)
    SELECT parent_asin, features_json, details_json, product_title
    FROM products 
    WHERE parent_asin IS NOT NULL;
""")
conn.commit()
print("FTS table created and populated successfully.")

import spacy
nlp = spacy.load("en_core_web_sm")
def extract_entities(s):
    """Extract named entities from a string using spaCy."""
    doc = nlp(s)
    entities = []
    for ent in doc.ents:
        if ent.label_ in ['CARDINAL', 'ORDINAL']:
            continue
        entities.append(ent.text)
    if len(entities) == 0:
        return [s]
    return entities

Creating FTS5 virtual table 'products_fts'...
Clearing FTS table to prevent duplicates...
FTS table created and populated successfully.


In [16]:
import spacy
# --- Target Selectivities ---
target_selectivities_pct = [0.1, 1.0, 10.0, 50.0]
filter_results = []

# --- Main Analysis Loop ---
total_query_products = len(df_products_of_interest)
print(f"Total products in database: {total_products_in_db}")
print(f"Iterating through {total_query_products} query products...")

all_outputs = []

for r, row in tqdm(df_products_of_interest.iterrows(), total=total_query_products):
    asin = row['parent_asin']
    output_record = row.to_dict()
    output_record['filters'] = []
    for col, config in filter_columns_config.items():
        # print(col)
        value = row.get(col)
        if is_value_empty(value):
            continue

        if config['type'] == 'exact_match':
            where_clause = f"LOWER({col}) = LOWER(?)"
            params = (str(value),)
            filter_value_for_log = value
            
            stats = get_filter_stats(cursor, where_clause, params, asin, total_products_in_db)
            # print(stats)
            if stats:
                output_record['filters'].append({
                    'filter_column': col,
                    'filter_value': filter_value_for_log,
                    'total_in_db': total_products_in_db,
                    **stats
                })
        if config['type'] == 'NER_contains':
            text = row[col]
            if is_value_empty(text): continue
            
            entities = extract_entities(text)

            for entity in entities:

                # FTS match
                where_clause = f"{col} MATCH ?"
                # escape any double quotes *inside* the value by doubling them.
                escaped_val = entity.replace('"', '""')           
                params = ('"' + escaped_val + '"',)   
                stats = get_filter_stats(cursor, where_clause, params, asin, total_products_in_db, use_fts=True)
                # print(stats)
                if stats:
                    output_record['filters'].append({
                        'filter_column': col,
                        'filter_value': entity,
                        'total_in_db': total_products_in_db,
                        **stats
                    })
        elif config['type'] == 'range':
            num_value = pd.to_numeric(value, errors='coerce')
            if pd.isna(num_value):
                continue
            
            sorted_values = distributions.get(col)
            if sorted_values is None or len(sorted_values) == 0:
                continue # Skip if pre-fetch failed or column is empty

            # Find the position of our product's value in the sorted array
            current_value_idx = np.searchsorted(sorted_values, num_value)
            
            # Use the distribution of this column, not the total DB count
            total_values_for_col = len(sorted_values) 

            for target_pct in target_selectivities_pct:
                # 1. Convert percentage to target count
                target_count = int((target_pct / 100.0) * total_values_for_col)
                if target_count < 1:
                    target_count = 1
                
                # 2. Find range indices
                half_count = target_count // 4 # often fall out of range
                lower_idx = max(0, current_value_idx - half_count)
                upper_idx = min(total_values_for_col - 1, current_value_idx + half_count)

                # 3. Get values at indices to determine delta
                lower_bound_val = sorted_values[lower_idx]
                upper_bound_val = sorted_values[upper_idx]

                # 4. Calculate symmetric delta
                delta = max(num_value - lower_bound_val, upper_bound_val - num_value)
                
                # 5. Define range
                min_val = num_value - delta
                max_val = num_value + delta
                min_val = max(0, min_val) # Ensure non-negative
                if col == 'rating_number':
                    min_val, max_val = int(min_val), int(max_val)
                # 6. Validate this single, calculated delta with the DB
                where_clause = f"{col} BETWEEN ? AND ?"
                params = (min_val, max_val)
                
                stats = get_filter_stats(cursor, where_clause, params, asin, total_products_in_db)
                # print(stats)
                if stats:
                    output_record['filters'].append({
                        'filter_column': col,
                        'filter_value': (min_val, max_val),
                        'total_in_db': total_products_in_db,
                        **stats
                })

        elif config['type'] == 'json_contains':
            values_to_check = get_json_values(value)
            if not values_to_check:
                continue
            
            for val in values_to_check:
                # use FTS MATCH for json_contains
                where_clause = f"{col} MATCH ?"
                entities = extract_entities(val)
                # if col == 'features_json':
                #     print(val, entities)
                for entity in entities:
                    # escape any double quotes *inside* the value by doubling them.
                    escaped_val = entity.replace('"', '""')
                    # Wrap 'val' in double quotes to force a PHRASE search.
                    params = ('"' + escaped_val + '"',)     
                    stats = get_filter_stats(cursor, where_clause, params, asin, total_products_in_db, use_fts=True)

                    if stats:
                        output_record['filters'].append({
                            'filter_column': col,
                            'filter_value': entity,
                            'total_in_db': total_products_in_db,
                            **stats
                        })

    all_outputs.append(output_record)

Total products in database: 1033025
Iterating through 1352 query products...


  0%|          | 0/1352 [00:00<?, ?it/s]

Ineffective filter detected (0% selectivity) with SQL query: SELECT COUNT(*) FROM products_fts WHERE (product_title MATCH ?), params: ('"Fashion New High-end Men\'s Business Myopia Sun Photochromic Lens Metal Half Frame Optical Nearsighted Glasses (-100)"',)
Ineffective filter detected (0% selectivity) with SQL query: SELECT COUNT(*) FROM products_fts WHERE (features_json MATCH ?), params: ('"Cotton,Coral Silk"',)
Ineffective filter detected (0% selectivity) with SQL query: SELECT COUNT(*) FROM products_fts WHERE (details_json MATCH ?), params: ('"CSG05310BB"',)
Ineffective filter detected (0% selectivity) with SQL query: SELECT COUNT(*) FROM products_fts WHERE (product_title MATCH ?), params: ('"Christian Brands Sacred Heart Chaplet"',)
Ineffective filter detected (0% selectivity) with SQL query: SELECT COUNT(*) FROM products_fts WHERE (details_json MATCH ?), params: ('"RD047"',)
Ineffective filter detected (0% selectivity) with SQL query: SELECT COUNT(*) FROM products_fts WHERE (prod

In [17]:
# save all_outputs to a json file
with open('filters_all.json', 'w') as f:
    json.dump(all_outputs, f, indent=2)

# for each query, preserve only unique filters based on (match_count, match_percentage)
for output in all_outputs:
    filters = output['filters']
    unique_filters = {}
    for f in filters:
        key = (f['filter_column'], f['match_count'], f['match_percentage'])
        if key not in unique_filters:
            unique_filters[key] = f

    thresholds = [0.01, 0.1, 1.0, 10, 50, 70]
    filters = list(unique_filters.values())
    threshold_filters = {}
    for t in range(len(thresholds)-1):
        interval = (thresholds[t], thresholds[t+1])
        threshold_filters[interval] = len([f for f in filters if interval[0] < f['match_percentage'] <= interval[1]])
    
    print(threshold_filters)
    output['filters'] = list(unique_filters.values())

with open('filters_deduplicated.json', 'w') as f:
    json.dump(all_outputs, f, indent=2)

{(0.01, 0.1): 3, (0.1, 1.0): 1, (1.0, 10): 2, (10, 50): 2, (50, 70): 1}
{(0.01, 0.1): 2, (0.1, 1.0): 1, (1.0, 10): 4, (10, 50): 1, (50, 70): 0}
{(0.01, 0.1): 2, (0.1, 1.0): 0, (1.0, 10): 3, (10, 50): 3, (50, 70): 0}
{(0.01, 0.1): 1, (0.1, 1.0): 2, (1.0, 10): 2, (10, 50): 4, (50, 70): 0}
{(0.01, 0.1): 0, (0.1, 1.0): 3, (1.0, 10): 5, (10, 50): 2, (50, 70): 0}
{(0.01, 0.1): 2, (0.1, 1.0): 2, (1.0, 10): 4, (10, 50): 4, (50, 70): 0}
{(0.01, 0.1): 1, (0.1, 1.0): 1, (1.0, 10): 6, (10, 50): 1, (50, 70): 0}
{(0.01, 0.1): 2, (0.1, 1.0): 2, (1.0, 10): 4, (10, 50): 3, (50, 70): 0}
{(0.01, 0.1): 1, (0.1, 1.0): 4, (1.0, 10): 2, (10, 50): 3, (50, 70): 0}
{(0.01, 0.1): 2, (0.1, 1.0): 3, (1.0, 10): 6, (10, 50): 4, (50, 70): 0}
{(0.01, 0.1): 3, (0.1, 1.0): 1, (1.0, 10): 1, (10, 50): 5, (50, 70): 0}
{(0.01, 0.1): 4, (0.1, 1.0): 3, (1.0, 10): 1, (10, 50): 4, (50, 70): 0}
{(0.01, 0.1): 4, (0.1, 1.0): 3, (1.0, 10): 3, (10, 50): 3, (50, 70): 0}
{(0.01, 0.1): 2, (0.1, 1.0): 3, (1.0, 10): 4, (10, 50): 2, (50, 

In [ ]:
import spacy
from tqdm.notebook import tqdm
# Load the small, lightweight English model
nlp = spacy.load("en_core_web_sm")
for r, row in tqdm(df_products_of_interest.iterrows(), total=total_query_products):
    text = row['product_title']
    print(text)
    doc = nlp(text)

    # Print the entities it found
    for ent in doc.ents:
        if ent.label_ in ['CARDINAL', 'ORDINAL']:
            continue
        print(f"Text: {ent.text},  Label: {ent.label_}")

  0%|          | 0/1351 [00:00<?, ?it/s]

LKOUS Womens Casual Long Sleeves High Waist One Piece Cargo Ripped Jean Pant Demin Jumpsuits Romper Overall Playsuit
Text: LKOUS Womens,  Label: PERSON
Text: Jean Pant,  Label: PERSON
Fashion New High-end Men's Business Myopia Sun Photochromic Lens Metal Half Frame Optical Nearsighted Glasses (-100)
Girlie Girl Originals Wash Your Hands Coral Silk Short Sleeve T-Shirt (X-Large)
Text: Girlie Girl Originals,  Label: PERSON
NinaQueen "Star Pave" 925 Sterling Silver Purple Cubic Zirconia Bead Charms for Bracelet Necklace Jewelry Christmas Gifts For Women Birthday Anniversary Gifts For Her Teen Girls Wife Mom Daughter
Text: NinaQueen,  Label: PRODUCT
Text: 925,  Label: CARDINAL
Christian Brands Sacred Heart Chaplet
Text: Christian Brands Sacred Heart Chaplet,  Label: ORG
Kpop Stray Kids I AM You Hoodie Felix Jisung Jeongin Sweater Pullover Jacket
Text: Felix Jisung Jeongin Sweater,  Label: PERSON
Twist-O-Flex™ Band for the Fitbit Charge 3 & Charge 4 Silver / M
Text: the Fitbit Charge 3 & Ch

KeyboardInterrupt: 